# Decision Tree Classifier usando ID3

<p> Primeiramente vamos verificar como as árvores de decisão funcionam, suas vantagens e desvatagens: </p>

1. Como funciona:
<p> Uma árvore de decisão é como um uma árvore de fluxograma onde o nó interno representa o atributo e/ou característica, o galho representa a regra de decisão, e cada folha representa o resultado. Para termos uma interpretação mais fácil segue abaixo uma ilustração:</p>

![img](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1545934190/1_r5ikdb.png)

![img](https://insightimi.files.wordpress.com/2020/03/1.png?w=720)

<p> As ideias básicas atrás da árvore de decisão são:

&emsp; 1.1 - Usar o melhor atributo usando ASM (Attribute Selection Measures) para dividir os registros

&emsp; 1.2 - Transformar esse atributo em um nó de decisão e dividir o dataset em datasets menores

&emsp; 1.3 - Começar a montar a árvore por repetir esse processo recursivamente para cada criança até que uma condição seja encontrada:
<p>

<li> Todos os tuples pertencem ao mesmo valor </li>
<li> Não ter mais atributos sobrando</li>
<li>Não ter mais instancias</li>

</p>

![img](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1545934190/2_btay8n.png)




1. 4 - ASM (Attribute Selection Measures):
&emsp;<p>ASM é uma forma heurísitca para selecionar o critério de partição dos dados da melhor forma possível. Também sendo conhecido como regras de divisão por nos ajudar a determinar os pontos de ruptura para tuples em um dado node. Providenciando um rank para cada atributo e/ ou característica explicando o dataset dado. O atributo com a melhor nota será selecionado como o prórpio atributo de separação. No caso de atributos de dados contínuos, os pontos de de divisão para os galhos também precisam ser difinidos. As seleções mais populares são Ganha, Razão Ganha e Índice Gini.


&emsp;&emsp; &emsp; 1.4.1 - Ganho de informação: Shannon inventou o conceito de entropia, que mede a impureza de um dado dataset. Na física e matemática, entropia é referida como sendo a impureza de um dataset de entrada. O algoritmo árvore de decisão ID3 (Iterative Dichotomiser) usa o ganho de informação:

![img](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1545934190/3_tvqfga.png)

&emsp;&emsp; &emsp; Onde, Pi é a probabilidade que um tuple arbitrário D pertenca a classe Ci.

![img](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1545934190/4_vvrzww.png)

![img](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1545934190/5_trlrj8.png)

&emsp;&emsp; &emsp; Onde: 

&emsp;&emsp; &emsp; &emsp; Info(D) é a média de informações necessárias para identificar a rótulo de classe de um tuple em D.

&emsp;&emsp; &emsp; &emsp; |Dj| / |Di| age como peso na partição de número j.

&emsp;&emsp; &emsp; &emsp; InfoA(D) é a informação necessária para classificar  tuple de um D baseado na partição por A.

&emsp;&emsp; &emsp; 1.4.2 - Razão Ganha: tem um viés para um atributo com muitos desfechos. Isso que significa que prefere o atributo com o maior número de valores distintos. 

&emsp;&emsp; &emsp; 1.4.3 - Índice Gini: Outro algoritmo de árvore de decisão usa esse método, CART (Classification and Regression Tree). O Índice Gini considera uma divisão binária para cada partição.
</p>

## Pros

<li> Árvores de dicisão são fáceis de interpretar e visualizar</li>
<li> Pode captar facilmente padrões não lineares</li>
<li> Precisa de menos processamento de dados por parte do usuário</li>
<li> Pode ser usado em Feature Engineering como na prvisão de dados faltantes.</pi>

## Contras
<li> Sensível a "noisy data". Pode dar overfit com "noisy data".</li>
<li> Uma pequena variação nos dados pode resultar uma diferente árvore de decisão</li><li>Árvores de decisão são influenciadas por dados não balanceados, então é recomedado que seja feito o balanceamento dos dados antes.</li>

# Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier,plot_tree

# Importando Dados

In [ ]:
df = pd.read_csv("data/data.csv", delimiter = "\t").rename(columns = {'<DATE>': 'date', '<OPEN>': 'open', '<HIGH>':'high', '<CLOSE>':'close', '<LOW>':'low'}).drop(columns = ['<TICKVOL>', '<VOL>', '<SPREAD>'])
df['adj_close'] = df['close'] * 0.8375918 # criando coluna de adf_close para futuros calculos

# Dfinindo indicador que será utilizado

In [ ]:
def rsi(data, column, window=1):
    """Retorn o calculo de RSI sobre o DF e coluna"""   
    
    data = data.copy()
    
    # Establish gains and losses for each day
    data["Variation"] = data[column].diff()
    data = data[1:]
    data["Gain"] = np.where(data["Variation"] > 0, data["Variation"], 0)
    data["Loss"] = np.where(data["Variation"] < 0, data["Variation"], 0)

    # Calculate simple averages so we can initialize the classic averages
    simple_avg_gain = data["Gain"].rolling(window).mean()
    simple_avg_loss = data["Loss"].abs().rolling(window).mean()
    classic_avg_gain = simple_avg_gain.copy()
    classic_avg_loss = simple_avg_loss.copy()

    for i in range(window, len(classic_avg_gain)):
        classic_avg_gain[i] = (classic_avg_gain[i] * (window - 1) + data["Gain"].iloc[i]) / window
        classic_avg_loss[i] = (classic_avg_loss[i] * (window - 1) + data["Loss"].abs().iloc[i]) / window
    
    # Calculate the RSI
    RS = classic_avg_gain / classic_avg_loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

In [ ]:
def rsi_table(df):
    """Usa um DF criando uma coluna com o sinal para o ML"""
    df["IFR2"] = rsi(df, column="adj_close")
    df["Target1"] = df["high"].shift(1)
    df["Target2"] = df["high"].shift(2)
    df["Target"] = df[["Target1", "Target2"]].max(axis=1)
    df.drop(columns=["Target1", "Target2"], inplace=True)
    
    # Define exact buy price
    rsi_parameter = 40
    df["buy_price"] = np.where(df["IFR2"] <= rsi_parameter, df["close"], 0)

    # Define exact sell price
    df["sell_price"] = np.where(
        df["high"] > df['Target'], 
        np.where(df['open'] > df['Target'], df['open'], df['Target']),
        0) 
    trade_logic = []
    for i in range(len(df['close'])):
        if ((df['buy_price'][i] == 0) & (df['sell_price'][i] == 0)) == True:
            trade_logic.append(-1)
        elif (df['buy_price'][i] != 0) == True:
            trade_logic.append(1)
        else:
            trade_logic.append(0)
    df['sinal'] = trade_logic
    df = df.drop(columns = {'IFR2', 'Target', 'buy_price', 'sell_price'})
    return df

# Definindo ID3

In [ ]:
class GadId3Classifier:
  def fit(self, input, output):
    data = input.copy()
    data[output.name] = output
    self.tree = self.decision_tree(data, data, input.columns, output.name)

  def predict(self, input):
    # converte a entrada em um dicionario de amostras
    samples = input.to_dict(orient='records')
    predictions = []

    # faz uma previsao para cada amostra
    for sample in samples:
      predictions.append(self.make_prediction(sample, self.tree, 1.0))

    return predictions

  def entropy(self, attribute_column):
    # encotnra valores unicos e a conta das suas frequencias para um dado atributo
    values, counts = np.unique(attribute_column, return_counts=True)

    # calcula a entropia para cada valor unico
    entropy_list = []

    for i in range(len(values)):
      probability = counts[i]/np.sum(counts)
      entropy_list.append(-probability*np.log2(probability))

    # calcula a soma de valor individual de entropia
    total_entropy = np.sum(entropy_list)

    return total_entropy

  def information_gain(self, data, feature_attribute_name, target_attribute_name):
    # encontra o total da entropia em subset dado
    total_entropy = self.entropy(data[target_attribute_name])

    # encontra valores unicos e a conta de suas frequencias para o atributo ser dividido
    values, counts = np.unique(data[feature_attribute_name], return_counts=True)

    # calcula o peso da entropia em um subset
    weighted_entropy_list = []

    for i in range(len(values)):
      subset_probability = counts[i]/np.sum(counts)
      subset_entropy = self.entropy(data.where(data[feature_attribute_name]==values[i]).dropna()[target_attribute_name])
      weighted_entropy_list.append(subset_probability*subset_entropy)

    total_weighted_entropy = np.sum(weighted_entropy_list)

    # calcula o ganho de informacao
    information_gain = total_entropy - total_weighted_entropy

    return information_gain

  def decision_tree(self, data, orginal_data, feature_attribute_names, target_attribute_name, parent_node_class=None):
    # casos base:
    # se os dados sao puros, retorna a classe majoritaria do subset
    unique_classes = np.unique(data[target_attribute_name])
    if len(unique_classes) <= 1:
      return unique_classes[0]
    # se o subset está vazio, sem amostras, retorna a classe majoritaria dos dados originais

    elif len(data) == 0:
      majority_class_index = np.argmax(np.unique(original_data[target_attribute_name], return_counts=True)[1])
      return np.unique(original_data[target_attribute_name])[majority_class_index]
    # se a os nao contem caracteristicas para treinar, retorna os pais da classe do no 
    elif len(feature_attribute_names) == 0:
      return parent_node_class
    # se nada acima e verdade, construa um galho:
    else:
      # determina os pais da classe do no atual 
      majority_class_index = np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])
      parent_node_class = unique_classes[majority_class_index]

      # determina o ganho de informacoes para cada carcteristica
      # escolhe a carcteristica que melhor divide os dados, ex. maior valor
      ig_values = [self.information_gain(data, feature, target_attribute_name) for feature in feature_attribute_names]
      best_feature_index = np.argmax(ig_values)
      best_feature = feature_attribute_names[best_feature_index]

      # cria a estrutura da arvore, vazio em primeiro momento
      tree = {best_feature: {}}

      # remove a melhor caracteristica das disponiveis, vai virar os pais do no
      feature_attribute_names = [i for i in feature_attribute_names if i != best_feature]

      # cria no abaixo do no pai
      parent_attribute_values = np.unique(data[best_feature])
      for value in parent_attribute_values:
        sub_data = data.where(data[best_feature] == value).dropna()

        # chama o algoritmo recursivamente
        subtree = self.decision_tree(sub_data, orginal_data, feature_attribute_names, target_attribute_name, parent_node_class)

        # adiciona uma subarvore para a arvore original
        tree[best_feature][value] = subtree

      return tree

  def make_prediction(self, sample, tree, default=1):
    # mapeia o sample data para a arvore
    for attribute in list(sample.keys()):
      # verificar se a caracteristica existe na arvore
      if attribute in list(tree.keys()):
        try:
          result = tree[attribute][sample[attribute]]
        except:
          return default

        result = tree[attribute][sample[attribute]]

        # se mais atributos existem no resultado, recursivamente encontra o melhor resultado
        if isinstance(result, dict):
          return self.make_prediction(sample, result)
        else:
          return result

In [ ]:
df = rsi_table(df)
X = df.drop(columns=["sinal", 'date'])
y = df["sinal"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3378, random_state = 42)

id3 = GadId3Classifier()
id3.fit(X_train, y_train)
y_pred = id3.predict(X_test)
accuracy_score(y_test, y_pred)*100

In [ ]:
    
# Initializing MT5 connection 
def input_mt5(x_1, x_2, x_3 = 1000):
    """Importa os dados do MetaTrader conforme solicitacao do usuario
    Parametros
    __________
    :param x_1: str, moeda que deseja importar
    :param x_2: str, timeframe que deseja utilizar conforme MQL5
    :param x_3: int, quantia de linhas que deseja importar, padrao igual a 1000
    __________
    retorna: dataframe contendo open, high, low, close, adj_close"""
    import MetaTrader5 as mt
    mt.initialize()
    mt.terminal_info()

    print("Inicialização bem sucedida: {}".format(mt.initialize()))
    # Copying data to pandas data frame
    stockdata = pd.DataFrame()
    rates = mt.copy_rates_from_pos(x_1, x_2,0, x_3)
    # Deinitializing MT5 connection
    mt.shutdown()
    print("Desligamento bem sucedido: {}".format(mt.shutdown()))
    stockdata['open'] = [y['open'] for y in rates]
    stockdata['high'] = [y['high'] for y in rates]
    stockdata['low'] = [y['low'] for y in rates]
    stockdata['close'] = [y['close'] for y in rates]
    stockdata['adj_close'] = stockdata.close * 0.8375918 
    
    return stockdata

In [ ]:
def buy(x, _vol):
    """Compra conforme parametros inseridos
    Parametros
    __________
    :param x: str, moeda que esta sendo utilizada
    :param _vol: float, volume da moeda conforme o mercado e a quantia desejada
    __________
    Retorna: compra dentro do MetaTrader5
    """
    price = mt.symbol_info_tick(x).ask
    
    sl  = price - 80.0*mt.symbol_info(x).point
    tp  = price + 100.0*mt.symbol_info(x).point

    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": x,
        "volume": _vol,
        "type": mt.ORDER_TYPE_BUY,
        "sl": sl,
        "tp": tp,
        "magic": 124512,
        "deviation": 0,
        "comment": "Buy Order",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }

    result = mt.order_send(request)
    print(x)
    print(f'OrderSended buy: {result}')

In [ ]:
def sell(x, _vol):
    """Venda conforme parametros inseridos
    Parametros
    __________
    :param x: str, moeda que esta sendo utilizada
    :param _vol: float, volume da moeda conforme o mercado e a quantia desejada
    __________
    Retorna: vende dentro do MetaTrader5
    """
    price = mt.symbol_info_tick(x).bid

    sl  = price + 80.0*mt.symbol_info(x).point
    tp  = price - 100.0*mt.symbol_info(x).point

    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": x,
        "volume": _vol,
        "type": mt.ORDER_TYPE_SELL,
        "sl": sl,
        "tp": tp,
        "magic": 124512,
        "deviation": 0,
        "comment": "Sell Order",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }

    result = mt.order_send(request)

    print(f'OrderSended sell: {result}')

In [ ]:
def main (x_1, x_2, x_3, x_4, x_5, x_6):
    """Itera dentro do MetaTrader5 com um tempo estipulado, realizando compras e vendas de acordo com a previsao do modelo
    Parametros
    __________
    :param x_1: int, valor em minutos para iterar
    :param x_2: int, tempo entre cada operacao
    :param x_3: str, moeda que deseja operar
    :param x_4: str, timeframe da operacao
    :param x_5: int, numero de linhas para importar para analise
    :param x_6: float, volume da moeda conforme o mercado e a quantia desejada
    __________
    retorna: a compra/venda dentro do MetaTrader5"""
    
    # Definicao de variavel para armazenar o tempo de operacao indicado e nao deixar ocorrer um loop: x_1
    var = x_1 * 60
    from time import sleep
    # While loop para rodar a logica junto ao modelo treinado e o MetaTrader5
    while var != 0:
        # Tempo que o algoritmo ira esperar entre cada iteracao (tempo de operacao no mercado): x_2
        sleep(x_2)
        last_price = input_mt5(x_3, x_4, x_5)
        predict_result = id3.predict(last_price)
        mt.initialize()
        
        if mt.positions_get(symbol = x_3) == ():
            if predict_result[-1] == 1 :
                #buy(x_3, x_6)
                print("compra")
                print(predict_result[-1], '\n')
                mt.shutdown()
                var -= x_2
                
            elif predict_result[-1] == 0 :
                #sell(x_3, x_6)
                print("venda")
                mt.shutdown()
            
            else: 
                print('nao pode dar predict')
                            
        else:
            print('Erro')
            break